In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# Import libraries
import os
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
import numpy as np
import rasterio
from rasterio.transform import from_origin

pd.set_option('display.max_columns', None)

In [2]:
# set data dir
DATA_DIR = Path('C:/Users/Reinhard/Documents/computer_vision_projects/amini_canopy_crop_challenge')
os.listdir(DATA_DIR)

['amini-canopy-or-crop-challenge.ipynb',
 'Fundamentals_of_RS.pdf',
 'introduction_to_remote_sensing.ipynb',
 'Notebook.ipynb',
 'processing_Earth_Obsersation_Data_for_Machine_Learning.ipynb',
 'SampleSubmission.csv',
 'StarterNotebook (2).ipynb',
 'Test.csv',
 'Train.csv',
 'Webinar_EO_series.ipynb',
 'Zindi_webiner_3.pdf']

In [3]:
# Load files
train = pd.read_csv(DATA_DIR / 'Train.csv')
test = pd.read_csv(DATA_DIR / 'Test.csv')
sample_submission = pd.read_csv(DATA_DIR / 'SampleSubmission.csv')

# Preview train file
train.head(10)

,ID,time,Green,Blue,RED,NIR,SWIR1,SWIR2,Red_Edge,Aerosols,Red_Edge_2,Red_Edge_3,Red_Edge_4,Water_vapor,Cirrus,NDVI,NDMI,NDWI,CI,Target
0,1D_0000,2021-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,1D_0000,2021-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,1D_0000,2021-01-14,0.0845,0.1030,0.0583,0.1977,0.1152,0.0464,0.0742,0.1326,0.1682,0.2243,0.2500,0.0347,0.0007,0.544531,0.263663,-0.401134,0.120000,0.0
3,1D_0000,2021-01-19,0.1095,0.1357,0.0858,0.1596,0.0782,0.0330,0.0873,0.1620,0.1396,0.1736,0.1820,0.0260,0.0011,0.300733,0.342304,-0.186176,0.008666,0.0
4,1D_0000,2021-01-24,0.1128,0.1300,0.0919,0.2141,0.1240,0.0608,0.1035,0.1546,0.1845,0.2302,0.2606,0.0703,0.0041,0.399346,0.266489,-0.309881,0.059365,0.0
5,1D_0000,2021-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
6,1D_0000,2021-02-03,0.1081,0.1284,0.0867,0.2044,0.1266,0.0568,0.0952,0.1550,0.1750,0.2316,0.2481,0.0373,0.0007,0.404328,0.235045,-0.308160,0.046729,0.0
7,1D_0000,2021-02-08,0.1017,0.1234,0.0791,0.2122,0.1318,0.0570,0.0923,0.1504,0.1852,0.2395,0.2572,0.0420,0.0037,0.456917,0.233721,-0.352023,0.077013,0.0
8,1D_0000,2021-02-13,0.0936,0.1138,0.0681,0.1962,0.1217,0.0534,0.0863,0.1507,0.1786,0.2327,0.2617,0.0366,0.0006,0.484677,0.234350,-0.354037,0.117876,0.0
9,1D_0000,2021-02-18,0.1270,0.1483,0.0997,0.2381,0.1409,0.0641,0.1142,0.1663,0.1988,0.2488,0.2874,0.0422,0.0054,0.409710,0.256464,-0.304300,0.067789,0.0


In [5]:
train.shape

(7558740, 20)

View on row of the data as an RGB image, what happens if you add more layers to this image? Does this give you an idea of how tabular data can be turned into image data and vice versa?

In [6]:
# Preview test file
test.head()

,ID,time,Green,Blue,RED,NIR,SWIR1,SWIR2,Red_Edge,Aerosols,Red_Edge_2,Red_Edge_3,Red_Edge_4,Water_vapor,Cirrus,NDVI,NDMI,NDWI,CI
0,1D_0005,2021-01-04,0.1199,0.1391,0.0978,0.2329,0.1782,0.0989,0.1146,0.2127,0.2074,0.2675,0.3081,0.0503,0.0011,0.408527,0.133058,-0.320295,0.079096
1,1D_0005,2021-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1D_0005,2021-01-14,0.0818,0.1010,0.0563,0.1848,0.1139,0.0459,0.0756,0.1324,0.1687,0.2246,0.2513,0.0372,0.0007,0.532974,0.237362,-0.386347,0.146323
3,1D_0005,2021-01-19,0.1270,0.1581,0.1031,0.1899,0.1186,0.0583,0.1124,0.1763,0.1639,0.1987,0.2156,0.0298,0.0012,0.296246,0.231118,-0.198485,0.043155
4,1D_0005,2021-01-24,0.1122,0.1311,0.0916,0.2125,0.1259,0.0608,0.1060,0.1553,0.1840,0.2307,0.2611,0.0709,0.0046,0.397567,0.255910,-0.308901,0.072874


In [7]:
test.shape

(4946879, 19)

In [7]:
# Preview sample submission
sample_submission.head()

,ID,Target
0,1D_0005,0
1,1D_000A,0
2,1D_000D,0
3,1D_000E,0
4,1D_0015,0


In [8]:
# Check shapes of files
train.shape, test.shape, sample_submission.shape

((7558740, 20), (4946879, 19), (16960, 2))

In [9]:
# Check for duplicates and missing values - train
train.duplicated().any(), train.isnull().sum().any() # Handle missing values by filling them with meadian or any other techniques

(False, True)

In [10]:
# Check for duplicates and missing values - test
test.duplicated().any(), test.isnull().sum().any() # Handle missing values by filling them with meadian or any other techniques

(False, True)

In [11]:
# Check unique ids per dataset
train.ID.nunique(), test.ID.nunique()

(26190, 16960)

In [12]:
# A single ID represents a unique pixel over different time periods
# create a targets_df

train_targets_df = train.groupby('ID')['Target'].first().reset_index()
train_targets_df.head()

,ID,Target
0,1D_0000,0.0
1,1D_0001,0.0
2,1D_0002,0.0
3,1D_0003,0.0
4,1D_0004,0.0


In [13]:
# Check data distribution
train_targets_df.Target.value_counts(normalize = True)

Target
0.0    0.454716
2.0    0.349752
1.0    0.195533
Name: proportion, dtype: float64

In [ ]:
train.head()

,ID,time,Green,Blue,RED,NIR,SWIR1,SWIR2,Red_Edge,Aerosols,Red_Edge_2,Red_Edge_3,Red_Edge_4,Water_vapor,Cirrus,NDVI,NDMI,NDWI,CI,Target
0,1D_0000,2021-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1D_0000,2021-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,1D_0000,2021-01-14,0.0845,0.1030,0.0583,0.1977,0.1152,0.0464,0.0742,0.1326,0.1682,0.2243,0.2500,0.0347,0.0007,0.544531,0.263663,-0.401134,0.120000,0
3,1D_0000,2021-01-19,0.1095,0.1357,0.0858,0.1596,0.0782,0.0330,0.0873,0.1620,0.1396,0.1736,0.1820,0.0260,0.0011,0.300733,0.342304,-0.186176,0.008666,0
4,1D_0000,2021-01-24,0.1128,0.1300,0.0919,0.2141,0.1240,0.0608,0.1035,0.1546,0.1845,0.2302,0.2606,0.0703,0.0041,0.399346,0.266489,-0.309881,0.059365,0


In [14]:
# Aggrgate pixels over the different time periods to get a wholesome view
# Get columns to aggregate
agg_cols = train.columns.tolist()[2:-1]
agg_cols

['Green',
 'Blue',
 'RED',
 'NIR',
 'SWIR1',
 'SWIR2',
 'Red_Edge',
 'Aerosols',
 'Red_Edge_2',
 'Red_Edge_3',
 'Red_Edge_4',
 'Water_vapor',
 'Cirrus',
 'NDVI',
 'NDMI',
 'NDWI',
 'CI']

What are these bands? Are there spefic bands used choose over another?

## Feature Engineering

What features or aggregations would you create?

Consider the timeseries component of the data

## Modeling

What is the best model or models you can you use for this type of problem?

Is there only one model? Would an ensemble of models add something?

What happens if you take into consideration run size and model implementation, what would be the lightest, fastest and most accurate model?

## Making your submission

Here is your code to help you prepare the data for a submission to Zindi. Note you will need to edit this code to the dataframe names you used in your modeling building.

In [ ]:
# Prepare test data
X_test = test.drop(['ID'], axis = 1)

# Make predictions on test
test_preds = model.predict(X_test)

# Prepares sub file
sub = pd.DataFrame()
sub['ID'] = test.ID
sub['Target'] = test_preds
sub.head()

,ID,Target
0,1D_0005,0
1,1D_000A,0
2,1D_000D,0
3,1D_000E,0
4,1D_0015,0


In [ ]:
# Create submission to be uploaded to Zindi for scoring
sub.to_csv('Submission.csv', index = False)

## Best of luck!

Remember their is only learning on Zindi!